In [1]:
from IPython.display import display, HTML
import scipy.constants as sc
from scipy.optimize import fsolve
from ipywidgets import interact, widgets
%matplotlib notebook

import numpy as np
import cmath
import matplotlib.pyplot as plt
import scipy.constants as sc
plt.rcParams.update({
    "text.usetex": True,
    'text.latex.preamble': r"\usepackage{amsmath} \boldmath"
})
import os
import h5py

print(os.getpid())
%cd ../

34241
/media/work/docs/codes/QuCF/scripts-py


In [2]:
# --- Launch to change cells' width ---
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [3]:
import pylib.mix as mix
import pylib.qucf_read as qucf_r
import pylib.qucf_oracle as qucf_o
path_ = "../simulations/tests/test-simple-matrix"
file_name_oracle_ = "circuit_OH"

In [4]:
# ---------------------------------------------------------------
# --- Read the matrix ---
# ---------------------------------------------------------------
mix.reload_module(qucf_r)

# dd4 = qucf_r.read_matrix(path_, "test_n4_OUTPUT.hdf5") 
# dd5 = qucf_r.read_matrix(path_, "test_n5_OUTPUT.hdf5") 
# dd6 = qucf_r.read_matrix(path_, "test_n6_OUTPUT.hdf5") 
# dd7 = qucf_r.read_matrix(path_, "test_n7_OUTPUT.hdf5") 
dd8 = qucf_r.read_matrix(path_, "test_n8_OUTPUT.hdf5") 

Reading the matrix from: test_n8_OUTPUT.hdf5
from the path: ../simulations/tests/test-simple-matrix
date of the simulation:  06-27-2023 11:17:07
matrix name:  U_BE
N = 256


In [5]:
# --------------------------------------------------------------
# --- Chose the case ---
# --------------------------------------------------------------
dd = dict(dd8)

# Form the matrix:
N = dd["N"]
A = np.zeros((N,N), dtype=complex)


a = 1.2
b = 10.4
f = 8.2
c = 4.6

A[0, 1] = b*1j;    A[0, 2] = a
A[1, 0] = b*1j;    A[1, 2] = f;         A[1, 3] = 0.40
A[2, 0] = a;    A[2, 1] = f;         A[2, 3] = c + f*1j; A[2, 4] = 0.3
A[3, 1] = 0.40; A[3, 2] = c + f*1j;  A[3, 4] = c + f*1j; A[3, 5] = 0.3
for i in range(4, N-4):
    A[i, i-2] = 0.3; A[i, i-1] = c + f*1j; A[i, i+1] = c + f*1j;   A[i,i+2] = 0.3
kk = N-4
A[kk, kk-2] = 0.3; A[kk, kk-1] = c + f*1j; A[kk, kk+1] = c + f*1j; A[kk, kk+2] = 0.4
kk = N-3
A[kk, kk-2] = 0.3; A[kk, kk-1] = c + f*1j; A[kk, kk+1] = 0.15;     A[kk, kk+2] = 0.3
kk = N-2
A[kk, kk-2] = 0.4; A[kk, kk-1] = 0.15;       A[kk, kk+1] = 0.10; 
kk = N-1
A[kk, kk-2] = 0.3; A[kk, kk-1] = 0.10;    

# --- Normalize the matrix and define the circuit structure ---
mix.reload_module(mix)
mix.reload_module(qucf_o)

D = dd["A-real"]
A_norm = qucf_o.normalize_matrix_A(A, D)

# Define the circuit structure (problem-specific part):
oo_circ = qucf_o.Circuit__()
anc = qucf_o.Regs__()
inp = qucf_o.Regs__()
anc.add_reg("ar", dd["regs"]["ar"], True)
inp.add_reg("r",  dd["regs"]["r"], False)
oo_circ.set_regs(inp, anc)
oo_circ.compute_N_registers()

amin.(excl. zero) value in D: 		2.500e-01
amax. value in A: 			1.040e+01
amin.(excl. zero) value in A: 		1.000e-01

amax. value in A-norm: 			1.954e-01
amin. (excl. zero)  value in A-norm: 	1.879e-03
Adding a register 'ar'...
Adding a register 'r'...


In [6]:
# -----------------------------------------------
# --- Computation of oracle angles ---
# -----------------------------------------------
mix.reload_module(mix)
mix.reload_module(qucf_o)

# 1. Compute angles of rotations gates:
Ty, Tc_ampl, Tc_phase = qucf_o.compute_angles(A_norm, D)

# 2. Rearrange angles:
grid_Ry       = qucf_o.create_grid_of_gates(oo_circ, Ty)
grid_Rc_ampl  = qucf_o.create_grid_of_gates(oo_circ, Tc_ampl)
grid_Rc_phase = qucf_o.create_grid_of_gates(oo_circ, Tc_phase)

# 3. Create groups of angles:
flag_neighbor = True

# create original groups:
groups_Ry = qucf_o.create_groups(oo_circ, grid_Ry,      grid_R_phase=None,          flag_neighbor=flag_neighbor)
groups_Rc = qucf_o.create_groups(oo_circ, grid_Rc_ampl, grid_R_phase=grid_Rc_phase, flag_neighbor=flag_neighbor)

sys_gates_ = qucf_o.SystemGates__(oo_circ, D, groups_Ry, groups_Rc)

# sort groups:
sys_gates_.sort_gates_groups()

# extend the gates:
sys_gates_.extend_gates_groups()

# 4 Construct the circuit:
sys_gates_.construct_circuit_OH(path_, file_name_oracle_)

N-input-registers = 1 with [8] qubits
N-ancilla-registers = 1 with [3] qubits
Total number of input qubits: 8
Matrix size that can be encoded: 256
N-gates: 42


In [7]:
# --- Recreate the matrix using the groups ---
qucf_o.compare_matrices(oo_circ, sys_gates_.reconstruct_matrix_using_groups(), A_norm)

Matrix size: {:d} 256
Min. abs. nonzero in A-real: 1.879e-03
Min. abs. nonzero in A-imag: 1.541e-01

Max. abs. diff-real: 3.903e-17
Max. abs. diff-imag: 0.000e+00


In [8]:
# --- Count the number of groups ---
mix.reload_module(qucf_o)
sys_gates_.count_groups()

sys_gates_.print_angles("Ry")


N = 256
--- Groups for the gate Ry ---
Number of groups: 16
--- Groups for the gate Rc ---
Number of groups: 4


Group angles for the gate Ry
Section: 1
3.013965503367e+00; 3.099075935352e+00; 3.096496439258e+00; 3.099075935352e+00; 3.109706165777e+00; 
Section: 2
4.469569286585e+00; 3.164139327730e+00; 3.130964224572e+00; 
Section: 4
1.813616020595e+00; 3.119045979450e+00; 3.130964224572e+00; 
Section: 5
3.013965503367e+00; 3.099075935352e+00; 3.096496439258e+00; 3.099075935352e+00; 3.109706165777e+00; 


In [7]:
# ----------------------------------------------------------
# --- Read results ---
# ----------------------------------------------------------
mix.reload_module(qucf_r)

dd_res = qucf_r.read_matrix(path_, "simple-matrix_OUTPUT.hdf5") 

qucf_o.compare_matrices(oo_circ, dd_res["A"], A_norm)

Reading the matrix from: simple-matrix_OUTPUT.hdf5
from the path: ../simulations/tests/test-simple-matrix
date of the simulation:  06-28-2023 13:52:23
matrix name:  U_BE
N = 256
Min. abs. nonzero in A-real: 1.879e-03
Min. abs. nonzero in A-imag: 1.541e-01

Max. abs. diff-real: 7.651e-14
Max. abs. diff-imag: 4.274e-14


In [ ]:
# --------------------------------------------------------------
# --- Plot matrix D and A-real---
# --------------------------------------------------------------
mix.reload_module(qucf_r)
figD     = qucf_r.plot_matrix(dd["A-mask"], fontsize = 20, cmap = "bwr")
figA_real = qucf_r.plot_matrix(np.real(A), fontsize = 20, cmap = "bwr")

In [7]:
# ---------------------------------------------------------------
# --- Function to print matrix subblocks ---
# ---------------------------------------------------------------
mix.reload_module(mix)
def print_submatrix(B, flag_imag = False):
    N = B.shape[0]

    
    print("\n")
    print("****** N: {:d} *************************************".format(N))
    
    max_val = np.max(np.max(np.abs(B)))
    print("max|A|: {:0.3e}".format(max_val))
    
    min_val = np.min(np.min(np.abs(B[np.nonzero(B)])))
    print("min|A|: {:0.3e}".format(min_val))
    
    # --------------------------------------------------
    Nr_print, Nc_print = N, N
    ir_start, ic_start = 0, 0
    
    B_plot = np.imag(B) if flag_imag else np.real(B)
    line_desr = "Imag part" if flag_imag else "Real part"


    print("-------------------------- {:s} -----------------------------------------".format(line_desr))
    mix.print_subblock_colored(
        B_plot, 
        ir_start, ic_start,
        Nr_print, Nc_print,
        ff=[6, 3, "f"], 
        n_in_row = Nc_print, 
        gap_be = " ", 
    )

In [8]:
# ------------------------------------------
# --- Print matrix D ---
# ------------------------------------------
print_submatrix(D) 



****** N: 32 *************************************
max|A|: 3.536e-01
min|A|: 2.500e-01
-------------------------- Real part -----------------------------------------
--- 0:   0.000  0.354  0.354  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 1:   0.354  0.000  0.250  0.354  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 2:   0.354 -0.250  0.000  0.250  0.250  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 3:   0.000  0.354  0.250  0.000  0.250  0.250  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0

In [9]:
# ------------------------------------------
# --- Print normalized original matrix A ---
# ------------------------------------------
print_submatrix(A_norm, False) 
print_submatrix(A_norm, True) 



****** N: 32 *************************************
max|A|: 1.000e-01
min|A|: 2.500e-02
-------------------------- Real part -----------------------------------------
--- 0:   0.000  0.030  0.057  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 1:   0.030  0.000  0.100  0.100  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 2:   0.057  0.100  0.000  0.000  0.075  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000
--- 3:   0.000  0.100  0.000  0.000  0.000  0.075  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0

In [29]:
ff_line = "{:0." + "{:d}".format(-int(np.log10(1e-12))) + "e}"
ff_line.format(1.259376e-6)

'1.259376000000e-06'

In [30]:
xx = [10, 20]

yy = xx
print(yy)

xx = yy

xx = [30, 40]
print(yy)

[10, 20]
[10, 20]


In [33]:
if np.abs(10 -np.nan) < 1e-12:
    print("same")
else:
    print("not same")

not same


In [40]:
print(np.any(np.isnan([np.nan, 20])))
print(np.all(np.isnan([np.nan, 20])))

True
False


In [39]:
np.any(np.isnan(np.nan))

True

In [41]:
"{:19s}".format("data")

'data               '

In [35]:
xx = np.array([
    [10, 20, 30],
    [100, 200, 300]
])
sqrt_sum = np.sqrt(np.sum(np.abs(xx)**2,axis=1))
print(sqrt_sum)
np.max(sqrt_sum)

[ 37.417 374.166]


374.16573867739413